In [63]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from genericpath import exists
import statistics
import scipy.stats as st

In [64]:
def process_df_for_gantt(df: pd):

    worker_dict = {}

    for row in df.itertuples(index=True, name="Pandas"):
        task_id = str(row.task_id)
        start = row.start_time
        end = row.end_time
        duration = end - start
        host_ip = row.host_ip
        pid = row.pid
        task = row.file_name + "/" + row.column_name
        worker_dict[task_id] = {
            "start": start,
            "end": end,
            "duration": duration,
            "host_ip": host_ip,
            "pid": pid,
            "task": task,
        }

    return worker_dict

In [65]:
def analyze_signle_local_logs_file(
    logs_file_path_name: str = None,
    initial_process_time: float = 0,
    total_process_time: float = 0,
    instance_number:int = 0 ,
    num_unfinished_tasks: int = 0,
    code_templates:str = None,

) -> dict:

    if exists(logs_file_path_name) is False:
        print(f"{logs_file_path_name} does not exist")
        return
    df = pd.read_csv(logs_file_path_name)

    error_task = df[(df["alert_type"] == "SYSTEM_ERROR")]
    num_error_task = len(error_task)

    worker_dict = process_df_for_gantt(df)

    shortest_task = ""
    longest_task = ""
    min_duration = float("inf")
    max_duration = 0
    tasks_durtaion_sum = 0
    average_task_duration = 0
    total_tasks = 0
    min_start = float("inf")
    max_end = 0
    duration = 0
    task_duration_in_parallelism = 0
    efficiency = 0
    for key, value in worker_dict.items():
        # logger.info(f"ip {value['host_ip']} ")
        # logger.info(f"ip {value} ")
        if ("start" in value) is False:
            # logger.warning(f"missing 'start' key in task {key}")
            print(f"missing 'start' key in task {key}")
            continue
        if ("end" in value) is False:
            # logger.warning(f"missing 'end' key in task {key}")
            print(f"missing 'end' key in task {key}")
            continue
        start = float(value["start"])
        end = float(value["end"])
        duration = value["duration"]
        task = value["task"]
        if task == "loop_tasks_status_task":
            continue
        tasks_durtaion_sum += duration
        if start < min_start:
            min_start = start
        if end > max_end:
            max_end = end

        if duration < min_duration:
            min_duration = duration
            shortest_task = task
        if duration > max_duration:
            max_duration = duration
            longest_task = task
        total_tasks += 1
    task_duration_in_parallelism = max_end - min_start
    if total_tasks > 0:
        average_task_duration = tasks_durtaion_sum / total_tasks
    else:
        average_task_duration = tasks_durtaion_sum

    if task_duration_in_parallelism > 0:
        efficiency = int(
            (
                (tasks_durtaion_sum - task_duration_in_parallelism)
                / task_duration_in_parallelism
            )
            * 100
        )

    ip_info = dict()

            #     "start": start,
            # "end": end,
            # "duration": duration,
            # "host_ip": host_ip,
            # "pid": pid,
            # "task": task,
    
    # --------------------------
    # calcuate effeciency factor
    # --------------------------
    # step 1 , accumulate the process time of each host_ip/pid
    for key, value in worker_dict.items():
        _duration = float(value["duration"])
        # _start = float(value["start"])
        # _end = float(value["end"])
        _host_ip = value["host_ip"]
        _pid = value["pid"]
        
        ip = str(_host_ip) + "/" + str(_pid)
        if ip in ip_info:
            ip_info[ip][key] = value
        else:
            ip_info[ip] = {
                key: value
            }
    print("===========\n")
    for key , value in ip_info.items():
        _task_min_stast  = float("inf")
        _task_max_end = 0
        _accu_duration = 0
        for taskid, taskvalue in value.items():
            _start = taskvalue['start']
            _end = taskvalue['end']
            _accu_duration += taskvalue['duration']
            if _task_min_stast >_start:
                _task_min_stast = _start
            if _task_max_end < _end:
                _task_max_end = _end
        ip_info[key]['min_start'] = float(_task_min_stast)
        ip_info[key]['max_end'] = float(_task_max_end)
        ip_info[key]['accumulate_durations'] =float(_accu_duration)
        _efficiency  = _accu_duration  / (_task_max_end -_task_min_stast)
        print("------------------")
        print(key,_task_min_stast,  _task_max_end,_accu_duration,  _efficiency)
        print("------------------")
        ip_info[key]['efficiency'] = round(_efficiency*100, 2)


    print("===========\n")
        # if ip in ip_accumulation_druations:
        #     # druation
        #     ip_accumulation_druations[ip]['duration'] += _duration
           
        # else:
        #     ip_accumulation_druations[ip]['duration'] = _duration


    # for key, value in ip_accumulation_druations.items():
    #     print("-------------------")
    #     print(f"key :{key}, value :{value}")




    # accumulated_process_duration = 0
    # for key, value in ip_accumulation_druations.items():
    #     accumulated_process_duration += value
    # effeciency_of_each_ip_pid = dict()
    # effeciencyFactor = 0
    # if task_duration_in_parallelism > 0:

    #     for key, value in ip_accumulation_druations.items():
    #         effeciency_of_each_ip_pid[key] = value / task_duration_in_parallelism

    # if len(ip_accumulation_druations) > 0:
    #     effeciencyFactor = accumulated_process_duration * 100 / (
    #         task_duration_in_parallelism * len(ip_accumulation_druations)
    #     )


    # messages_rate = 0
    # if task_duration_in_parallelism > 0: 
    #     messages_rate = 3*total_tasks / task_duration_in_parallelism
    # # normalizing_process_time = task_duration_in_parallelism / (total_tasks*average_task_duration) 
    # performance_dict = {
    #     "file": logs_file_path_name,
    #     "instance_number": instance_number,
    #     "code_templates": code_templates,
    #     "total_tasks": total_tasks,
    #     "average_task_duration": round(average_task_duration, 2),
    #     "min_duration": round(min_duration, 2),
    #     "max_duration": round(max_duration, 2),
    #     "num_error_task": num_error_task,
    #     "longest_task": longest_task,
    #     "shortest_task": shortest_task,
    #     "num_unfinished_tasks": num_unfinished_tasks,
    #     "task_duration_in_parallelism": round(task_duration_in_parallelism, 2),
    #     "tasks_durtaion_sum": round(tasks_durtaion_sum, 2),
    #     "initial_process_time": round(initial_process_time, 2),
    #     "total_process_time": round(total_process_time, 2),
    #     "effeciencyFactor": round(effeciencyFactor, 2),
    #     "messages_rate": round(messages_rate, 2),
    #     # "normalizing_process_time": round(normalizing_process_time,2),
    #     # "optimal_normalizing_process_time": round(1/instance_number,2),
    # }
    # return performance_dict

In [66]:
logs_folder_list = ['./gcd-solar-644-mixed/solardatatools-32']
index = 0
logs_files_list = []

# process log file and convert to performance dict
performance = {}
number_of_message_per_task = 3
for _folder in logs_folder_list:
    index = 0 
    base, path = os.path.split(_folder)
    # print(base, path)
    code_templates, number_instance = path.split("-")
    # print(code_templates,number_instance)
    _per_list = []
    _task_duration_in_parallelism_list = []
    _effeciencyFactor_list = []
    _average_task_duration_list = []
    _messages_rate_list = []
    _normalizing_process_time_list = []
    # _optimal_normalizing_process_time_list = []
    total_tasks = 1
    for _file in os.listdir(_folder):
        filename,extension = os.path.splitext(_file)
        if extension == ".csv":
            _name_comp = filename.split("-")
            if len(_name_comp) > 1 and _name_comp[0] == "logs": 
                logs_file_path_name  = _folder +"/"+_file
                per_dict = analyze_signle_local_logs_file(
                    logs_file_path_name=logs_file_path_name,
                    instance_number = number_instance,
                    initial_process_time=0,
                    total_process_time=0,
                    num_unfinished_tasks=0,
                    code_templates = code_templates,
                )
                # print(per_dict)


------------------
192.168.69.30/15 0.7765743536725433
------------------
------------------
192.168.2.137/14 0.22926415330504546
------------------
------------------
192.168.2.137/15 0.6632045141519548
------------------
------------------
192.168.60.214/15 0.7107537059350568
------------------
------------------
192.168.53.143/15 0.7156191581650723
------------------
------------------
192.168.25.205/15 0.7089678388283991
------------------
------------------
192.168.43.48/13 0.5769161317368933
------------------
------------------
192.168.2.110/15 0.6609361132066982
------------------
------------------
192.168.38.77/15 0.678969410836423
------------------
------------------
192.168.43.48/14 0.7547936946355437
------------------
------------------
192.168.45.119/15 0.6662903708135299
------------------
------------------
192.168.79.224/15 0.7153901418949464
------------------
------------------
192.168.17.65/15 0.7376381044895828
------------------
------------------
192.168.82.12